In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from treelib import Tree
import operator
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from scipy.stats import t as t_dist
from scipy.stats import norm, chi2
import pickle

In [2]:
class Node:
    def __init__(self, name, feature=None, threshold=None, left_child=None, right_child=None, is_leaf=False, value=-1, depth=-1):
        self.feature = feature
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child
        self.is_leaf = is_leaf
        self.value = value
        self.name = name
        self.depth = depth

In [97]:
class MyDecisionTree:
    def __init__(self, min_samples=1, max_depth=-1, max_thresholds=None, min_unique_values=None, num_random_columns=None):
        self.root_node = Node('root', depth=0)
        self.node_count = 0
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.max_thresholds = max_thresholds
        self.min_unique_values = min_unique_values
        self.num_random_columns = num_random_columns
    
    def predict(self, X, cols_d):
        
        Y_pred = []
        for i in range(len(X)):

            x_i = X[i]
            # print(x_i[0])
            curr_node = self.root_node
            predicted_y = np.array(self.predict_util(x_i, curr_node, cols_d))
            # print(predicted_y)
            Y_pred.append(predicted_y)

        return np.array(Y_pred)

    def predict_util(self, x, curr_node, cols_d):
        
        # print(curr_node.feature)
        if(curr_node.is_leaf):
            return curr_node.value
        if x[cols_d[curr_node.feature]] <= curr_node.threshold:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.left_child, cols_d)
        else:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.right_child, cols_d)

    def fit(self, X, Y):
        self.fit_util(X, Y, self.root_node)


    def fit_util(self, X, Y, current_node):
        if np.unique(Y).shape[0] == 1:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        scores = []
        num_random_columns = len(X.columns)
        if self.num_random_columns is not None:
            num_random_columns = self.num_random_columns
        random_columns = X.columns.to_numpy()[np.random.permutation(len(X.columns))[:num_random_columns]]

        if(self.num_random_columns is None):
            random_columns = X.columns
        # print(random_columns)
        for column in random_columns:
            X_train = X[column].to_numpy()
            X_train = X_train.reshape(X_train.shape[0], 1)
            clf = LogisticRegression(random_state=0, max_iter=100)
            clf.fit(X_train, Y)
            # pred_Y = clf.predict(X_train)
            # precision = precision_score(pred_Y, Y)
            score = clf.score(X_train, Y)
            # print(precision)
            scores.append(score)
            
        scores = np.array(scores)
        best_feature = X.columns[np.argmax(scores)]
        best_feature_values = np.unique(X[best_feature])
        best_feature_values.sort()

        partition = self.get_partition(X, Y, best_feature, best_feature_values, current_node.depth)
        
        if partition == None:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        (X_left, Y_left), (X_right, Y_right), threshold = partition
        current_node.threshold = threshold
        current_node.feature = best_feature
        current_node.name = f'{best_feature} {threshold} {self.node_count}'
        self.node_count += 1
        current_node.left_child = Node('No name', depth=current_node.depth + 1)
        current_node.right_child = Node('No name', depth=current_node.depth + 1)

        self.fit_util(X_left, Y_left, current_node.left_child)
        self.fit_util(X_right, Y_right, current_node.right_child)

        

    def do_split(self, X, thresh):
        """
            Split the data at a node based on threshold
        """

        left_child_ids = np.where(X <= thresh, True, False)
        right_child_ids = np.where(X > thresh, True, False)
        return left_child_ids, right_child_ids
    
    def find_entropy(self, Y):
        probs = []
        possible_classes, counts = np.unique(Y, return_counts=True)
        sort_indices = np.argsort(possible_classes)
        possible_classes = possible_classes[sort_indices]
        counts = counts[sort_indices]
        
        for class_label, count in zip(possible_classes, counts):
            probs.append(count/Y.shape[0])
        
        entropy = 0
        for prob in probs:
            entropy -= prob*np.log2(prob)
        
        return entropy

    def get_partition(self, X, Y, feature, thresholds, current_node_depth):
        '''
            This function should return left and right
            partitions according to appropritate
            partitioning algorithm. Return None if all
            data has same label
        '''
        
        # if only 1 class available at a node OR MIN_SAMPLES left at a node - Leaf Node reached
        if(len(Y) < self.min_samples):
            return None
        if(current_node_depth == self.max_depth):
            return None
        
        best_info_gain = -float('inf')
        best_thresh = thresholds[0]      
        threshold_cnt = len(thresholds)  
        if self.max_thresholds is not None:
            threshold_cnt = self.max_thresholds
        min_unique_values = len(thresholds)

        if self.min_unique_values is not None:
            min_unique_values = self.min_unique_values
        
        print("Now thresholding!")
        for thresh in (thresholds[:-1] if len(thresholds) <= min_unique_values else thresholds[:-1:len(thresholds) // threshold_cnt]):

            left_child_ids, right_child_ids = self.do_split(X[feature].to_numpy(), thresh)

            parent_pts = X.shape[0]
            left_child_pts = len(left_child_ids)
            right_child_pts = len(right_child_ids)

            info_gain = self.find_entropy(Y) - (left_child_pts / parent_pts) * self.find_entropy(Y[left_child_ids]) - (right_child_pts / parent_pts) * self.find_entropy(Y[right_child_ids])

            if(info_gain > best_info_gain):

                best_info_gain = info_gain
                best_thresh = thresh
        
        # partition according to best threshold
        best_left_ids, best_right_ids = self.do_split(X[feature].to_numpy(), best_thresh)
        # print(len(Y[best_left_ids]), len(Y[best_right_ids]))
        if(len(Y[best_left_ids]) == 0 or len(Y[best_right_ids]) == 0):
            # print(len(Y[best_left_ids]), len(Y[best_right_ids]))
            return None
        return (X[best_left_ids], Y[best_left_ids]), (X[best_right_ids], Y[best_right_ids]), best_thresh
    
    def print_tree(self):
        tree = Tree()
        self.print_tree_util(self.root_node, tree)
        tree.show()
        return tree

    def print_tree_util(self, root, tree, parent=None):
        if parent is not None:
            tree.create_node(root.name, root.name, parent=parent.name)
        else:
            tree.create_node(root.name, root.name)
        if root.is_leaf:
            return
        self.print_tree_util(root.left_child, tree, root)
        self.print_tree_util(root.right_child, tree, root)


In [175]:
class Node2:
    def __init__(self, name, feature=None, threshold=None, left_child1=None, left_child2=None, right_child1=None, right_child2=None, is_leaf=False, value=-1, depth=-1):
        self.feature = feature
        self.threshold = threshold
        self.left_child1 = left_child1
        self.left_child2 = left_child2
        self.right_child1 = right_child1
        self.right_child2 = right_child2
        self.is_leaf = is_leaf
        self.value = value
        self.name = name
        self.depth = depth
    

In [176]:
class MyDecisionTree2:
    def __init__(self, min_samples=1, max_depth=-1, max_thresholds=None, min_unique_values=None, num_random_columns=None):
        self.root_node = Node2('root', depth=0)
        self.node_count = 0
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.max_thresholds = max_thresholds
        self.min_unique_values = min_unique_values
        self.num_random_columns = num_random_columns
    
    def predict(self, X, cols_d):
        
        Y_pred = []
        for i in range(len(X)):

            x_i = X[i]
            # print(x_i[0])
            curr_node = self.root_node
            predicted_y = np.array(self.predict_util(x_i, curr_node, cols_d))
            # print(predicted_y)
            Y_pred.append(predicted_y)

        return np.array(Y_pred)

    def predict_util(self, x, curr_node, cols_d):
        
        if(curr_node.is_leaf):
            return curr_node.value
        if x[cols_d[curr_node.feature[0]]] <= curr_node.threshold[0] and x[cols_d[curr_node.feature[1]]] <= curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.left_child1, cols_d)

        if x[cols_d[curr_node.feature[0]]] > curr_node.threshold[0] and x[cols_d[curr_node.feature[1]]] <= curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.left_child2, cols_d)

        if x[cols_d[curr_node.feature[0]]] <= curr_node.threshold[0] and x[cols_d[curr_node.feature[1]]] > curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.right_child1, cols_d)

        elif x[cols_d[curr_node.feature[0]]] > curr_node.threshold[0] and x[cols_d[curr_node.feature[1]]] > curr_node.threshold[1]:
            if curr_node.is_leaf:
                return curr_node.value
            return self.predict_util(x, curr_node.right_child2, cols_d)

    def fit(self, X, Y):
        self.fit_util(X, Y, self.root_node)


    def fit_util(self, X, Y, current_node):
        if np.unique(Y).shape[0] == 1:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        scores = {}
        num_random_columns = len(X.columns)
        if self.num_random_columns is not None:
            num_random_columns = self.num_random_columns
        random_columns = X.columns.to_numpy()[np.random.permutation(len(X.columns))[:num_random_columns]]
        if(self.num_random_columns is None):
            random_columns = X.columns
        for column1 in random_columns:
            for column2 in random_columns:
                column = [column1, column2]
                X_train = X[column].to_numpy()
                X_train = X_train.reshape(X_train.shape[0], 2)
                clf = LogisticRegression(random_state=0)
                clf.fit(X_train, Y)
                score = clf.score(X_train, Y)
                scores[(column1, column2)] = score
            

        scores_sorted = dict( sorted(scores.items(), key=operator.itemgetter(1),reverse=True))
        best_feature_set = list(scores_sorted.keys())[0]
        best_feature1_values = np.unique(X[best_feature_set[0]])
        best_feature2_values = np.unique(X[best_feature_set[1]])

        best_feature1_values.sort()
        best_feature2_values.sort()

        partition = self.get_partition(X, Y, best_feature_set[0], best_feature_set[1], best_feature1_values, best_feature2_values, current_node.depth)
        
        if partition == None:
            current_node.is_leaf = True
            unq, counts = np.unique(Y, return_counts=True)
            max_freq_idx = np.argmax(counts).flatten()
            current_node.value = unq[max_freq_idx].squeeze()
            current_node.name = f'leaf {current_node.value} {self.node_count}'
            self.node_count += 1
            return
            
        (X_left1, Y_left1), (X_left2, Y_left2), (X_right1, Y_right1), (X_right2, Y_right2), threshold1, threshold2 = partition

        threshold = [threshold1, threshold2]
        # print("Thresholds: ",threshold)
        current_node.threshold = threshold
        current_node.feature = best_feature_set
        current_node.name = f'{best_feature_set} {threshold} {self.node_count}'
        self.node_count += 1
        current_node.left_child1 = Node2('unnamed', depth=current_node.depth + 1)
        current_node.left_child2 = Node2('unnamed', depth=current_node.depth + 1)
        current_node.right_child1 = Node2('unnamed', depth=current_node.depth + 1)
        current_node.right_child2 = Node2('unnamed', depth=current_node.depth + 1)

        unq, counts = np.unique(Y, return_counts=True)
        max_freq_idx = np.argmax(counts).flatten()
        if(X_left1.shape[0] == 0):
            current_node.left_child1.is_leaf = True
            current_node.left_child1.value = unq[max_freq_idx].squeeze()
            current_node.left_child1.name = f'leaf {current_node.left_child1.value} {self.node_count}'
            self.node_count+=1 
            # return
        
        if(X_left2.shape[0] == 0):
            current_node.left_child2.is_leaf = True
            current_node.left_child2.value = unq[max_freq_idx].squeeze()
            current_node.left_child2.name = f'leaf {current_node.left_child2.value} {self.node_count}'
            self.node_count+=1 
            # return
        
        if(X_right1.shape[0] == 0):
            current_node.right_child1.is_leaf = True
            current_node.right_child1.value = unq[max_freq_idx].squeeze()
            current_node.right_child1.name = f'leaf {current_node.right_child1.value} {self.node_count}'
            self.node_count+=1   
            # return     
        
        if(X_right2.shape[0] == 0):
            current_node.right_child2.is_leaf = True
            current_node.right_child2.value = unq[max_freq_idx].squeeze()
            current_node.right_child2.name = f'leaf {current_node.right_child2.value} {self.node_count}'
            self.node_count+=1  
            # return                  
          
        if(X_left1.shape[0] != 0):
            self.fit_util(X_left1, Y_left1, current_node.left_child1)
        
        if(X_left2.shape[0] != 0):
            self.fit_util(X_left2, Y_left2, current_node.left_child2)
        
        if(X_right1.shape[0] != 0):
            self.fit_util(X_right1, Y_right1, current_node.right_child1)
        
        if(X_right2.shape[0] != 0):
            self.fit_util(X_right2, Y_right2, current_node.right_child2)


    def do_split(self, X, thresh):
        """
            Split the data at a node based on threshold
        """

        left_child_ids = np.where(X <= thresh, True, False)
        right_child_ids = np.where(X > thresh, True, False)
        return left_child_ids, right_child_ids

    def do_split_final(self, X1, X2, thresh1, thresh2):

        """
            Split according to the best thresholds for the 2 features
        """   

        left1_ids = np.where(np.logical_and(X1 <= thresh1, X2 <= thresh2), True, False)
        left2_ids = np.where(np.logical_and(X1 > thresh1, X2 <= thresh2), True, False)
        right1_ids = np.where(np.logical_and(X1 <= thresh1, X2 > thresh2), True, False)
        right2_ids = np.where(np.logical_and(X1 > thresh1, X2 > thresh2), True, False)

        return left1_ids, left2_ids, right1_ids, right2_ids

    def find_entropy(self, Y):
        probs = []
        possible_classes, counts = np.unique(Y, return_counts=True)
        sort_indices = np.argsort(possible_classes)
        possible_classes = possible_classes[sort_indices]
        counts = counts[sort_indices]
        
        for class_label, count in zip(possible_classes, counts):
            probs.append(count/Y.shape[0])
        
        entropy = 0
        for prob in probs:
            entropy -= prob*np.log2(prob)
        
        return entropy
    
    def find_best_thresh_info_gain(self, thresholds1, thresholds2, X, Y, feature1, feature2):
        """
            This function finds the best threshold and info gain for a given feature
        """
        best_info_gain = -float('inf')
        best_thresh1 = thresholds1[0]   
        best_thresh2 = thresholds2[0]     
        threshold_cnt_1 = len(thresholds1)
        threshold_cnt_2  = len(thresholds2)
        if self.max_thresholds is not None:
            threshold_cnt_1, threshold_cnt_2 = self.max_thresholds, self.max_thresholds
        min_unique_values_1 = len(thresholds1)
        min_unique_values_2 = len(thresholds2)

        if self.min_unique_values is not None:
            min_unique_values_1, min_unique_values_2 = self.min_unique_values, self.min_unique_values
        
        print("Now thresholding!")
        for thresh1 in (thresholds1[:-1] if len(thresholds1) <= min_unique_values_1 else thresholds1[:-1:len(thresholds1) // threshold_cnt_1]):
            for thresh2 in (thresholds2[:-1] if len(thresholds2) <= min_unique_values_2 else thresholds2[:-1:len(thresholds2) // threshold_cnt_2]):
                left_child1_ids, left_child2_ids,  right_child1_ids, right_child2_ids = self.do_split_final(X[feature1].to_numpy(),X[feature2].to_numpy(), thresh1, thresh2)

                parent_pts = X.shape[0]
                left_child1_pts = len(left_child1_ids)
                left_child2_pts = len(left_child2_ids)
                right_child1_pts = len(right_child1_ids)
                right_child2_pts = len(right_child2_ids)

                info_gain = self.find_entropy(Y) - (left_child1_pts / parent_pts) * self.find_entropy(Y[left_child1_ids]) - (left_child2_pts / parent_pts) * self.find_entropy(Y[left_child2_ids]) - (right_child1_pts / parent_pts) * self.find_entropy(Y[right_child1_ids]) - (right_child2_pts / parent_pts) * self.find_entropy(Y[right_child2_ids])
                
                if(info_gain > best_info_gain):

                    best_info_gain = info_gain
                    best_thresh1 = thresh1
                    best_thresh2 = thresh2

        
        return best_thresh1, best_thresh2, best_info_gain

    def get_partition(self, X, Y, feature1, feature2, thresholds1, thresholds2, current_node_depth):
        '''
            This function should return left and right
            partitions according to appropritate
            partitioning algorithm. Return None if all
            data has same label
        '''
        
        # if only 1 class available at a node OR MIN_SAMPLES left at a node - Leaf Node reached
        if(len(Y) < self.min_samples):
            return None
        if(current_node_depth == self.max_depth):
            return None
        
        best_thresh1, best_thresh2, best_info_gain = self.find_best_thresh_info_gain(thresholds1, thresholds2, X, Y, feature1, feature2)

        # partition according to best threshold
        best_left_ids1, best_left_ids2, best_right_ids1, best_right_ids2 = self.do_split_final(X[feature1].to_numpy(), X[feature2].to_numpy(), best_thresh1, best_thresh2)

        # print(len(Y[best_left_ids1]), len(Y[best_left_ids2]), len(Y[best_right_ids1]) , len(Y[best_right_ids2]))

        count_0 = 0
        if(len(Y[best_left_ids1]) == 0):
            count_0 += 1
        if(len(Y[best_left_ids2]) == 0):
            count_0 += 1
        if(len(Y[best_right_ids1]) == 0):
            count_0 += 1
        if(len(Y[best_right_ids2]) == 0):
            count_0 += 1

        if(count_0 == 3):
            print(len(Y[best_left_ids1]), len(Y[best_left_ids2]), len(Y[best_right_ids1]) , len(Y[best_right_ids2]))
            return None

        return (X[best_left_ids1], Y[best_left_ids1]), (X[best_left_ids2], Y[best_left_ids2]), (X[best_right_ids1], Y[best_right_ids1]),(X[best_right_ids2], Y[best_right_ids2]), best_thresh1, best_thresh2    
    
    def print_tree(self):
        tree = Tree()
        self.print_tree_util(self.root_node, tree)
        tree.show()
        return tree

    def print_tree_util(self, root, tree, parent=None):
        if parent is not None:
            # print(root.name)
            tree.create_node(root.name, root.name, parent=parent.name)
        else:
            # print(root.name)
            tree.create_node(root.name, root.name)
        if root.is_leaf:
            return
        self.print_tree_util(root.left_child1, tree, root)
        self.print_tree_util(root.left_child2, tree, root)
        self.print_tree_util(root.right_child1, tree, root)
        self.print_tree_util(root.right_child2, tree, root)


In [101]:
# def paired_t_test(p):
#     p_hat = np.mean(p)
#     n = len(p)
#     den = np.sqrt(sum([(diff - p_hat)**2 for diff in p]) / (n - 1))
#     t = (p_hat * (n**(1/2))) / den
    
#     p_value = t_dist.sf(t, n-1)*2
    
#     return t, p_value

# def z_test(acc1, acc2, n):

#     z, p = proportion_difference(acc1, acc2, n_1 = n)
#     return z,p

# def mcnamer_test(Y_test, Y_pred1, Y_pred2):
#     b = sum(np.logical_and((Y_pred2 != Y_test), (Y_pred1 == Y_test)))
#     c = sum(np.logical_and((Y_pred1 != Y_test), (Y_pred2 == Y_test)))

#     chi_2 = ((np.abs(b-c)-1)**2)/(b+c)
#     p = chi2.sf(chi_2, 1)

#     return chi_2, p

In [144]:
df_cancer = pd.read_csv('./data/final_data2.csv')

In [117]:
# df_cancer.replace("?", np.NaN, inplace=True)
# df_cancer = df_cancer.fillna(df_cancer.median())
# df_cancer = df_cancer.apply(pd.to_numeric)

In [145]:
X_cancer = df_cancer.drop(['Unnamed: 0','Biopsy', 'STDs:cervical condylomatosis', 'STDs:AIDS'], axis=1)
Y_cancer = df_cancer['Biopsy'].to_numpy()
X_train_cancer, X_test_cancer, Y_train_cancer, Y_test_cancer = train_test_split(X_cancer, Y_cancer, test_size=0.2, random_state = 42, stratify=Y_cancer)
print(X_train_cancer.shape)
print(np.unique(Y_train_cancer, return_counts=True))
print(np.unique(Y_test_cancer, return_counts=True))

(686, 31)
(array([0, 1], dtype=int64), array([642,  44], dtype=int64))
(array([0, 1], dtype=int64), array([161,  11], dtype=int64))


In [146]:
cols_cancer = X_cancer.columns
cols_d_cancer = {}
id=0
for col in cols_cancer:
    cols_d_cancer[col] = id
    id+=1

In [195]:
dt_cancer1 = MyDecisionTree()
dt_cancer1.fit(X_train_cancer, Y_train_cancer)
Y_pred1_cancer = dt_cancer1.predict(np.array(X_test_cancer.to_numpy()), cols_d_cancer)
accuracy1 = accuracy_score(Y_pred1_cancer, Y_test_cancer)
precision1 = precision_score(Y_pred1_cancer, Y_test_cancer)
recall1 = recall_score(Y_pred1_cancer, Y_test_cancer)
f1 = f1_score(Y_pred1_cancer, Y_test_cancer)
print("One-feature split accuracy: ", accuracy1)
print("One-feature split Precision: ", precision1)
print("One-feature split Recall: ", recall1)
print("One-feature split F1-Score: ", f1)

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresh

In [196]:
tree = dt_cancer1.print_tree()

Schiller 0 0
├── Age 13 1
│   ├── Age 70 3
│   │   ├── Age 52 4
│   │   │   ├── Age 50 5
│   │   │   │   ├── Age 49 6
│   │   │   │   │   ├── Age 48 7
│   │   │   │   │   │   ├── Age 46 8
│   │   │   │   │   │   │   ├── Age 45 9
│   │   │   │   │   │   │   │   ├── Age 44 10
│   │   │   │   │   │   │   │   │   ├── Age 14 11
│   │   │   │   │   │   │   │   │   │   ├── Age 43 13
│   │   │   │   │   │   │   │   │   │   │   ├── Age 42 14
│   │   │   │   │   │   │   │   │   │   │   │   ├── Age 41 15
│   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 40 16
│   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 39 17
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 38 18
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 37 19
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 36 20
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 35 21
│   │   │   │   │   │   │   │   │   │   │   │

In [197]:
tree.to_graphviz()

digraph tree {
	"Schiller 0 0" [label="Schiller 0 0", shape=circle]
	"Age 13 1" [label="Age 13 1", shape=circle]
	"STDs:syphilis 0.0 80" [label="STDs:syphilis 0.0 80", shape=circle]
	"Age 70 3" [label="Age 70 3", shape=circle]
	"leaf 0 2" [label="leaf 0 2", shape=circle]
	"IUD (years) 11.0 81" [label="IUD (years) 11.0 81", shape=circle]
	"leaf 0 148" [label="leaf 0 148", shape=circle]
	"Age 52 4" [label="Age 52 4", shape=circle]
	"leaf 0 79" [label="leaf 0 79", shape=circle]
	"Age 16 82" [label="Age 16 82", shape=circle]
	"leaf 0 147" [label="leaf 0 147", shape=circle]
	"Age 50 5" [label="Age 50 5", shape=circle]
	"leaf 0 78" [label="leaf 0 78", shape=circle]
	"Num of pregnancies 6.0 84" [label="Num of pregnancies 6.0 84", shape=circle]
	"leaf 0 83" [label="leaf 0 83", shape=circle]
	"Age 49 6" [label="Age 49 6", shape=circle]
	"leaf 0 77" [label="leaf 0 77", shape=circle]
	"Age 51 85" [label="Age 51 85", shape=circle]
	"leaf 0 146" [label="leaf 0 146", shape=circle]
	"Age 48 7" [label

In [198]:
pickle.dump(dt_cancer1, open("Results/Data1/1_Way/Models_Saved/Model.pkl", 'wb'))
metrics_array = np.array([accuracy1, precision1, recall1, f1])

with open('./Results/Data1/1_Way/Metrics/metrics.npy', 'wb') as f:
    np.save(f, metrics_array)

with open('./Results/Data1/1_Way/Y_Pred/Pred.npy', 'wb') as f:
    np.save(f, Y_pred1_cancer)


In [199]:
pickled_model = pickle.load(open("Results/Data1/1_Way/Models_Saved/Model.pkl", 'rb'))
Y_ = pickled_model.predict(np.array(X_test_cancer.to_numpy()), cols_d_cancer)
tree = pickled_model.print_tree()

Schiller 0 0
├── Age 13 1
│   ├── Age 70 3
│   │   ├── Age 52 4
│   │   │   ├── Age 50 5
│   │   │   │   ├── Age 49 6
│   │   │   │   │   ├── Age 48 7
│   │   │   │   │   │   ├── Age 46 8
│   │   │   │   │   │   │   ├── Age 45 9
│   │   │   │   │   │   │   │   ├── Age 44 10
│   │   │   │   │   │   │   │   │   ├── Age 14 11
│   │   │   │   │   │   │   │   │   │   ├── Age 43 13
│   │   │   │   │   │   │   │   │   │   │   ├── Age 42 14
│   │   │   │   │   │   │   │   │   │   │   │   ├── Age 41 15
│   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 40 16
│   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 39 17
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 38 18
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 37 19
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 36 20
│   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   ├── Age 35 21
│   │   │   │   │   │   │   │   │   │   │   │

In [211]:
acc_scores1 = []
precision_scores1 = []
recall_scores1 = []
f1_scores1 = []
strtfdKFold = StratifiedKFold(n_splits=5)
for train_index, test_index in strtfdKFold.split(X_cancer, Y_cancer):
    X_train, X_test = X_cancer.iloc[train_index, :], X_cancer.iloc[test_index, :]
    Y_train, Y_test = Y_cancer[train_index], Y_cancer[test_index]

    dt1_cancer = MyDecisionTree()
    dt1_cancer.fit(X_train, Y_train)
    Y_pred1_cancer = dt1_cancer.predict(np.array(X_test.to_numpy()), cols_d_cancer)

    accuracy1 = accuracy_score(Y_pred1_cancer, Y_test)
    precision1 = precision_score(Y_pred1_cancer, Y_test)
    recall1 = recall_score(Y_pred1_cancer, Y_test)
    f1 = f1_score(Y_pred1_cancer, Y_test)

    acc_scores1.append(accuracy1)
    precision_scores1.append(precision1)
    recall_scores1.append(recall1)
    f1_scores1.append(f1)

    print("----------")
    print("Accuracy1: ", accuracy1)
    print("Precision1: ", precision1)
    print("Recall1: ", recall1)
    print("F1-score: ", f1)
    print("----------")

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresh

In [212]:
with open("Results/Data1/1_Way/K_Folds/Accuracy/acc.npy",'wb') as f:
    np.save(f, np.array(acc_scores1))

with open("Results/Data1/1_Way/K_Folds/Precision/precision.npy", 'wb') as f:
    np.save(f, np.array(precision_scores1))

with open("Results/Data1/1_Way/K_Folds/Recall/recall.npy", 'wb') as f:
    np.save(f, np.array(recall_scores1))

with open("Results/Data1/1_Way/K_Folds/F1_Score/f1.npy", 'wb') as f:
    np.save(f, np.array(f1_scores1))

DATASET 1 - Cancer (2-features split)

In [200]:
dt_cancer2 = MyDecisionTree2()
dt_cancer2.fit(X_train_cancer, Y_train_cancer)

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
14 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
35 0 0 0
Now thresholding!
31 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
27 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresh

In [202]:
Y_pred2_cancer = dt_cancer2.predict(np.array(X_test_cancer.to_numpy()), cols_d_cancer)
accuracy2 = accuracy_score(Y_pred2_cancer, Y_test_cancer)
precision2 = precision_score(Y_pred2_cancer, Y_test_cancer)
recall2 = recall_score(Y_pred2_cancer, Y_test_cancer)
f1_2 = f1_score(Y_pred2_cancer, Y_test_cancer)
print("One-feature split accuracy: ", accuracy2)
print("One-feature split Precision: ", precision2)
print("One-feature split Recall: ", recall2)
print("One-feature split F1-Score: ", f1_2)

One-feature split accuracy:  0.9709302325581395
One-feature split Precision:  0.7272727272727273
One-feature split Recall:  0.8
One-feature split F1-Score:  0.761904761904762


In [209]:
pickle.dump(dt_cancer2, open("Results/Data1/2_Way/Models_Saved/Model.pkl", 'wb'))
metrics_array = np.array([accuracy2, precision2, recall2, f1_2])

with open("./Results/Data1/2_Way/Metrics/metrics.npy", 'wb') as f:
    np.save(f, metrics_array)

with open('./Results/Data1/2_Way/Y_Pred/Pred.npy', 'wb') as f:
    np.save(f, Y_pred2_cancer)


In [ ]:
acc_scores2 = []
precision_scores2 = []
recall_scores2 = []
f1_scores2 = []
strtfdKFold = StratifiedKFold(n_splits=5)
for train_index, test_index in strtfdKFold.split(X_cancer, Y_cancer):
    X_train, X_test = X_cancer.iloc[train_index, :], X_cancer.iloc[test_index, :]
    Y_train, Y_test = Y_cancer[train_index], Y_cancer[test_index]

    dt2_cancer = MyDecisionTree2()
    dt2_cancer.fit(X_train, Y_train)
    Y_pred2_cancer = dt2_cancer.predict(np.array(X_test.to_numpy()), cols_d_cancer)

    accuracy2 = accuracy_score(Y_pred2_cancer, Y_test)
    precision2 = precision_score(Y_pred2_cancer, Y_test)
    recall2 = recall_score(Y_pred2_cancer, Y_test)
    f1_2 = f1_score(Y_pred2_cancer, Y_test)

    acc_scores2.append(accuracy2)
    precision_scores2.append(precision2)
    recall_scores2.append(recall2)
    f1_scores2.append(f1_2)

    print("----------")
    print("Accuracy1: ", accuracy2)
    print("Precision1: ", precision2)
    print("Recall1: ", recall2)
    print("F1-score: ", f1_2)
    print("----------")

2.8200629894731226 0.04782506579017487


2nd Dataset - Fetal (For 1-feature split)

In [156]:
fetal_data = pd.read_csv('./data/fetal.csv')
fetal_data.head()

,Unnamed: 0,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,0,132,0.006,0.000,0.006,0.003,0.0,0.0,17,2.1,...,68,198,6,1,141,136,140,12,0,1
1,1,133,0.003,0.000,0.008,0.003,0.0,0.0,16,2.1,...,68,198,5,1,141,135,138,13,0,1
2,2,134,0.003,0.000,0.008,0.003,0.0,0.0,16,2.4,...,53,170,11,0,137,134,137,13,1,1
3,3,132,0.007,0.000,0.008,0.000,0.0,0.0,16,2.4,...,53,170,9,0,137,136,138,11,1,1
4,4,131,0.005,0.072,0.008,0.003,0.0,0.0,28,1.4,...,88,154,5,0,135,134,137,7,1,1


In [158]:
X_fetal = fetal_data.drop(['Unnamed: 0', 'fetal_health'], axis=1)
Y_fetal = fetal_data['fetal_health']

In [159]:
X_train_fetal, X_test_fetal, Y_train_fetal, Y_test_fetal = train_test_split(X_fetal, Y_fetal, test_size=0.2, random_state = 42, stratify=Y_fetal)
print(X_train_fetal.shape)
print(np.unique(Y_train_fetal, return_counts=True))
print(np.unique(Y_test_fetal, return_counts=True))

(1690, 21)
(array([1, 2, 3], dtype=int64), array([1316,  234,  140], dtype=int64))
(array([1, 2, 3], dtype=int64), array([330,  58,  35], dtype=int64))


In [160]:
cols_fetal = X_fetal.columns
cols_d_fetal = {}
id=0
for col in cols_fetal:
    cols_d_fetal[col] = id
    id+=1

In [194]:
dt_fetal1 = MyDecisionTree(max_depth=10)
dt_fetal1.fit(X_train_fetal, Y_train_fetal)
Y_pred1_fetal = dt_fetal1.predict(np.array(X_test_fetal.to_numpy()), cols_d_fetal)
accuracy1 = accuracy_score(Y_pred1_fetal, Y_test_fetal)
precision1 = precision_score(Y_pred1_fetal, Y_test_fetal, average='weighted')
recall1 = recall_score(Y_pred1_fetal, Y_test_fetal, average='weighted')
f1 = f1_score(Y_pred1_fetal, Y_test_fetal, average='weighted')
print("One-feature split accuracy: ", accuracy1)
print("One-feature split Precision: ", precision1)
print("One-feature split Recall: ", recall1)
print("One-feature split F1-Score: ", f1)

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
One-feature split accuracy:  0.8274231678486997
One-feature split Precision:  0.9638908033625567
One-feature split Recall:  0.8274231678486997
One-feature split F1-Score:  0.8839135393699566


In [192]:
tree = dt_fetal1.print_tree()

histogram_mean 89 0
├── leaf 3 1
└── percentage_of_time_with_abnormal_long_term_variability 73 2
    ├── leaf 3 798
    └── mean_value_of_short_term_variability 6.3 3
        ├── leaf 1 797
        └── mean_value_of_short_term_variability 3.3 4
            ├── histogram_mode 71 756
            │   ├── histogram_mode 90 758
            │   │   ├── baseline value 146 760
            │   │   │   ├── histogram_median 113 761
            │   │   │   │   ├── baseline value 115 763
            │   │   │   │   │   ├── baseline value 144 765
            │   │   │   │   │   │   ├── baseline value 120 766
            │   │   │   │   │   │   │   ├── baseline value 143 768
            │   │   │   │   │   │   │   │   ├── baseline value 142 769
            │   │   │   │   │   │   │   │   │   ├── baseline value 140 770
            │   │   │   │   │   │   │   │   │   │   ├── baseline value 138 771
            │   │   │   │   │   │   │   │   │   │   │   ├── baseline value 136 772
            │   │   │  

In [193]:
tree.to_graphviz()

digraph tree {
	"histogram_mean 89 0" [label="histogram_mean 89 0", shape=circle]
	"leaf 3 1" [label="leaf 3 1", shape=circle]
	"percentage_of_time_with_abnormal_long_term_variability 73 2" [label="percentage_of_time_with_abnormal_long_term_variability 73 2", shape=circle]
	"leaf 3 798" [label="leaf 3 798", shape=circle]
	"mean_value_of_short_term_variability 6.3 3" [label="mean_value_of_short_term_variability 6.3 3", shape=circle]
	"leaf 1 797" [label="leaf 1 797", shape=circle]
	"mean_value_of_short_term_variability 3.3 4" [label="mean_value_of_short_term_variability 3.3 4", shape=circle]
	"histogram_mode 71 756" [label="histogram_mode 71 756", shape=circle]
	"mean_value_of_short_term_variability 3.1 5" [label="mean_value_of_short_term_variability 3.1 5", shape=circle]
	"histogram_mode 90 758" [label="histogram_mode 90 758", shape=circle]
	"leaf 3 757" [label="leaf 3 757", shape=circle]
	"mean_value_of_long_term_variability 0.0 753" [label="mean_value_of_long_term_variability 0.0 753

DATASET 2 - FETAL (For 2-features split)

In [188]:
dt_fetal2 = MyDecisionTree2(max_depth=10)
dt_fetal2.fit(X_train_fetal, Y_train_fetal)

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

Now thresholding!


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

Now thresholding!


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

...

Now thresholding!


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

Now thresholding!
Now thresholding!
Now thresholding!


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:815: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/mod

Now thresholding!


In [189]:
Y_pred2_fetal = dt_fetal2.predict(np.array(X_test_fetal.to_numpy()), cols_d_fetal)
accuracy2 = accuracy_score(Y_pred2_fetal, Y_test_fetal)
precision2 = precision_score(Y_pred2_fetal, Y_test_fetal, average='weighted')
recall2 = recall_score(Y_pred2_fetal, Y_test_fetal, average='weighted')
f1_2 = f1_score(Y_pred2_fetal, Y_test_fetal, average='weighted')
print("One-feature split accuracy: ", accuracy2)
print("One-feature split Precision: ", precision2)
print("One-feature split Recall: ", recall2)
print("One-feature split F1-Score: ", f1_2)

One-feature split accuracy:  0.8416075650118203
One-feature split Precision:  0.9718701178715853
One-feature split Recall:  0.8416075650118203
One-feature split F1-Score:  0.895488495724234


DATASET 3 - BANKING (FOR 1-feature split)

In [164]:
df_banking = pd.read_csv("./data/pre-processed_banking.csv")

In [165]:
df_banking = pd.get_dummies(df_banking, columns=['job',
                                        'marital',
                                        'education',
                                        'housing',
                                        'loan',
                                        'contact',
                                        'month',
                                        'day_of_week',
                                        'poutcome'])

In [166]:
X_banking = df_banking.drop(['Unnamed: 0', 'y'], axis=1)
Y_banking = df_banking['y'].to_numpy()
X_train_banking, X_test_banking, Y_train_banking, Y_test_banking = train_test_split(X_banking, Y_banking, test_size=0.2, random_state=42, stratify=Y_banking)
print(X_train_banking.shape)

(32950, 60)


In [167]:
dt_banking1 = MyDecisionTree(max_depth=5, num_random_columns=30)
dt_banking1.fit(X_train_banking, Y_train_banking)

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!


In [170]:
cols_banking = X_banking.columns
cols_d_banking = {}
id=0
for col in cols_banking:
    cols_d_banking[col] = id
    id+=1

# Y_pred_banking

In [183]:
Y_pred1_banking = dt_banking1.predict(np.array(X_test_banking.to_numpy()), cols_d_banking)
accuracy1 = accuracy_score(Y_pred1_banking, Y_test_banking)
precision1 = precision_score(Y_pred1_banking, Y_test_banking)
recall1 = recall_score(Y_pred1_banking, Y_test_banking)
f1 = f1_score(Y_pred1_banking, Y_test_banking)
print("One-feature split accuracy: ", accuracy1)
print("One-feature split Precision: ", precision1)
print("One-feature split Recall: ", recall1)
print("One-feature split F1-Score: ", f1)

One-feature split accuracy:  0.887836853605244
One-feature split Precision:  0.005387931034482759
One-feature split Recall:  0.8333333333333334
One-feature split F1-Score:  0.010706638115631693


In [172]:
tree = dt_banking1.print_tree()

job_1 0 0
├── education_4 0 24
│   ├── euribor3m 0.634 25
│   │   ├── leaf 0 27
│   │   └── leaf 1 26
│   └── leaf 0 28
└── marital_2 0 1
    ├── duration 3509 15
    │   ├── leaf 1 23
    │   └── previous 4 16
    │       ├── campaign 39 17
    │       │   ├── leaf 0 18
    │       │   └── leaf 0 19
    │       └── cons_price_idx 93.876 20
    │           ├── leaf 1 21
    │           └── leaf 1 22
    └── job_6 0 2
        ├── age 23 10
        │   ├── age 26 12
        │   │   ├── leaf 0 13
        │   │   └── leaf 0 14
        │   └── leaf 0 11
        └── job_7 0 3
            ├── age 21 7
            │   ├── leaf 0 8
            │   └── leaf 0 9
            └── education_4 0 4
                ├── leaf 0 5
                └── leaf 0 6



In [173]:
tree.to_graphviz()

digraph tree {
	"job_1 0 0" [label="job_1 0 0", shape=circle]
	"education_4 0 24" [label="education_4 0 24", shape=circle]
	"marital_2 0 1" [label="marital_2 0 1", shape=circle]
	"euribor3m 0.634 25" [label="euribor3m 0.634 25", shape=circle]
	"leaf 0 28" [label="leaf 0 28", shape=circle]
	"duration 3509 15" [label="duration 3509 15", shape=circle]
	"job_6 0 2" [label="job_6 0 2", shape=circle]
	"leaf 0 27" [label="leaf 0 27", shape=circle]
	"leaf 1 26" [label="leaf 1 26", shape=circle]
	"leaf 1 23" [label="leaf 1 23", shape=circle]
	"previous 4 16" [label="previous 4 16", shape=circle]
	"age 23 10" [label="age 23 10", shape=circle]
	"job_7 0 3" [label="job_7 0 3", shape=circle]
	"campaign 39 17" [label="campaign 39 17", shape=circle]
	"cons_price_idx 93.876 20" [label="cons_price_idx 93.876 20", shape=circle]
	"age 26 12" [label="age 26 12", shape=circle]
	"leaf 0 11" [label="leaf 0 11", shape=circle]
	"age 21 7" [label="age 21 7", shape=circle]
	"education_4 0 4" [label="education_4 

DATASET 3 - BANKING (FOR 2-features split)

In [182]:
dt_banking2 = MyDecisionTree2(max_depth=5, num_random_columns=10)
dt_banking2.fit(X_train_banking, Y_train_banking)

Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
5 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
60 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
33 0 0 0
Now thresholding!
10813 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
49 0 0 0
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!
Now thresholding!


In [185]:
Y_pred2_banking = dt_banking2.predict(np.array(X_test_banking.to_numpy()), cols_d_banking)
accuracy2 = accuracy_score(Y_pred2_banking, Y_test_banking)
precision2 = precision_score(Y_pred2_banking, Y_test_banking)
recall2 = recall_score(Y_pred2_banking, Y_test_banking)
f1_2 = f1_score(Y_pred2_banking, Y_test_banking)
print("One-feature split accuracy: ", accuracy2)
print("One-feature split Precision: ", precision2)
print("One-feature split Recall: ", recall2)
print("One-feature split F1-Score: ", f1_2)

One-feature split accuracy:  0.8957271182325808
One-feature split Precision:  0.1853448275862069
One-feature split Recall:  0.6254545454545455
One-feature split F1-Score:  0.28595178719867004
